In [1]:
import pandas as pd
import numpy as np
import sklearn
from tqdm import tqdm 
import math

In [11]:
import session_info
session_info.show()

In [2]:
train_path = 'C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/imputed_data/final_imputed_train_x_station.gzip'
test_path = 'C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/imputed_data/final_imputed_test_x_station.gzip'
month_id_path = 'C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/Test/X_stations/Id_month_test.csv'

In [3]:
def Read_Parquet(chemin):
  return pd.read_parquet(chemin, engine='fastparquet')

In [6]:
train = Read_Parquet(train_path)
train.head()

,temp,precip,humidity,wind_direction,wind_speed,dew_point_temp,number_sta,date,Id,lat,lon,height_sta
index,,,,,,,,,,,,
0,279.28,0.0,91.4,200.0,3.05,277.97,14066001,2016-01-01 00:00:00,14066001_0_0,49.334,-0.431,2
1,278.76,0.0,91.4,190.0,2.57,277.45,14066001,2016-01-01 01:00:00,14066001_0_1,49.334,-0.431,2
2,278.27,0.0,91.7,181.0,2.26,277.02,14066001,2016-01-01 02:00:00,14066001_0_2,49.334,-0.431,2
3,277.98,0.0,93.0,159.0,2.62,276.95,14066001,2016-01-01 03:00:00,14066001_0_3,49.334,-0.431,2
4,277.32,0.0,95.9,171.0,2.99,276.72,14066001,2016-01-01 04:00:00,14066001_0_4,49.334,-0.431,2


In [7]:
train.dtypes

temp                     float64
precip                   float64
humidity                 float64
wind_direction           float64
wind_speed               float64
dew_point_temp           float64
number_sta                 int64
date              datetime64[ns]
Id                        object
lat                      float64
lon                      float64
height_sta                 int16
dtype: object

In [25]:
train.height_sta = train.height_sta.astype('int16')

In [6]:
test = Read_Parquet(test_path)
test.head()

,temp,precip,humidity,wind_direction,wind_speed,dew_point_temp,Id,number_sta,lat,lon,height_sta
index,,,,,,,,,,,
0,278.35,0.13,92.23,191.75,4.66,277.18,14047002_277_4,14047002,49.275,-0.712,60.0
1,278.40,0.00,91.18,184.79,4.79,277.06,14047002_277_5,14047002,49.275,-0.712,60.0
2,279.01,0.00,91.80,183.92,4.60,277.79,14047002_277_6,14047002,49.275,-0.712,60.0
3,279.66,0.00,93.41,179.11,5.17,278.65,14047002_277_7,14047002,49.275,-0.712,60.0
4,279.99,0.00,94.04,181.35,5.86,279.10,14047002_277_8,14047002,49.275,-0.712,60.0


In [7]:
test['Id'] = test['Id'].astype(str)

In [8]:
def spliting(string, index):
    liste = string.split("_")
    return liste[index]

In [9]:
test['number_sta'] = test['Id'].apply(lambda x : spliting(x, 0))
test['day_index'] = test['Id'].apply(lambda x : spliting(x, 1))
test['Hour'] = test['Id'].apply(lambda x : spliting(x, 2))

In [10]:
test.head()

,temp,precip,humidity,wind_direction,wind_speed,dew_point_temp,Id,number_sta,lat,lon,height_sta,day_index,Hour
index,,,,,,,,,,,,,
0,278.35,0.13,92.23,191.75,4.66,277.18,14047002_277_4,14047002,49.275,-0.712,60.0,277,4
1,278.40,0.00,91.18,184.79,4.79,277.06,14047002_277_5,14047002,49.275,-0.712,60.0,277,5
2,279.01,0.00,91.80,183.92,4.60,277.79,14047002_277_6,14047002,49.275,-0.712,60.0,277,6
3,279.66,0.00,93.41,179.11,5.17,278.65,14047002_277_7,14047002,49.275,-0.712,60.0,277,7
4,279.99,0.00,94.04,181.35,5.86,279.10,14047002_277_8,14047002,49.275,-0.712,60.0,277,8


In [ ]:
test.dtypes

temp              float64
precip            float64
humidity          float64
wind_direction    float64
wind_speed        float64
dew_point_temp    float64
Id                 object
number_sta         object
lat               float64
lon               float64
height_sta        float64
day_index          object
Hour               object
dtype: object

In [11]:
test['number_sta'] = test['Id'].apply(lambda line : spliting(line,0)).astype('int64')
test.head()

,temp,precip,humidity,wind_direction,wind_speed,dew_point_temp,Id,number_sta,lat,lon,height_sta,day_index,Hour
index,,,,,,,,,,,,,
0,278.35,0.13,92.23,191.75,4.66,277.18,14047002_277_4,14047002,49.275,-0.712,60.0,277,4
1,278.40,0.00,91.18,184.79,4.79,277.06,14047002_277_5,14047002,49.275,-0.712,60.0,277,5
2,279.01,0.00,91.80,183.92,4.60,277.79,14047002_277_6,14047002,49.275,-0.712,60.0,277,6
3,279.66,0.00,93.41,179.11,5.17,278.65,14047002_277_7,14047002,49.275,-0.712,60.0,277,7
4,279.99,0.00,94.04,181.35,5.86,279.10,14047002_277_8,14047002,49.275,-0.712,60.0,277,8


In [12]:
test.Hour = test.Hour.astype('int16')
test.height_sta = test.height_sta.astype('int16')

In [13]:
test.head()

,temp,precip,humidity,wind_direction,wind_speed,dew_point_temp,Id,number_sta,lat,lon,height_sta,day_index,Hour
index,,,,,,,,,,,,,
0,278.35,0.13,92.23,191.75,4.66,277.18,14047002_277_4,14047002,49.275,-0.712,60,277,4
1,278.40,0.00,91.18,184.79,4.79,277.06,14047002_277_5,14047002,49.275,-0.712,60,277,5
2,279.01,0.00,91.80,183.92,4.60,277.79,14047002_277_6,14047002,49.275,-0.712,60,277,6
3,279.66,0.00,93.41,179.11,5.17,278.65,14047002_277_7,14047002,49.275,-0.712,60,277,7
4,279.99,0.00,94.04,181.35,5.86,279.10,14047002_277_8,14047002,49.275,-0.712,60,277,8


In [15]:
month_id = pd.read_csv(month_id_path)
month_id.head()

,day_index,month
0,0,2
1,1,7
2,2,5
3,3,5
4,4,7


In [16]:
month_id = month_id.astype(str)

In [17]:
dictionnary = dict()
for idx, val in month_id.values:
  dictionnary[idx] = val

In [18]:
test['Month'] = test['day_index'].map(lambda x : dictionnary[x]).astype('int16')

In [19]:
test.head()

,temp,precip,humidity,wind_direction,wind_speed,dew_point_temp,Id,number_sta,lat,lon,height_sta,day_index,Hour,Month
index,,,,,,,,,,,,,,
0,278.35,0.13,92.23,191.75,4.66,277.18,14047002_277_4,14047002,49.275,-0.712,60,277,4,12
1,278.40,0.00,91.18,184.79,4.79,277.06,14047002_277_5,14047002,49.275,-0.712,60,277,5,12
2,279.01,0.00,91.80,183.92,4.60,277.79,14047002_277_6,14047002,49.275,-0.712,60,277,6,12
3,279.66,0.00,93.41,179.11,5.17,278.65,14047002_277_7,14047002,49.275,-0.712,60,277,7,12
4,279.99,0.00,94.04,181.35,5.86,279.10,14047002_277_8,14047002,49.275,-0.712,60,277,8,12


In [ ]:
test.dtypes

temp              float64
precip            float64
humidity          float64
wind_direction    float64
wind_speed        float64
dew_point_temp    float64
Id                 object
number_sta          int64
lat               float64
lon               float64
height_sta          int16
day_index          object
Hour                int16
Month               int16
dtype: object

## Feature engineering

In [26]:
train.columns

Index(['temp', 'precip', 'humidity', 'wind_direction', 'wind_speed',
       'dew_point_temp', 'number_sta', 'date', 'Id', 'lat', 'lon',
       'height_sta'],
      dtype='object')

In [27]:
train["Month"] = train["date"].dt.month.astype('int16')

In [28]:
train.dtypes

temp                     float64
precip                   float64
humidity                 float64
wind_direction           float64
wind_speed               float64
dew_point_temp           float64
number_sta                 int64
date              datetime64[ns]
Id                        object
lat                      float64
lon                      float64
height_sta                 int16
Month                      int16
dtype: object

## Getting the season from month

In [29]:
def season(month):
    if (month == 12 or 1 <= month <= 4):
        return "winter"   
    elif (4 <= month <= 5):
        return "spring" 
    elif (6 <= month <= 9):
        return "summer"
    else:
        return "fall"
    
train["season"] = train['Month'].apply(season)

In [30]:
train.head()

,temp,precip,humidity,wind_direction,wind_speed,dew_point_temp,number_sta,date,Id,lat,lon,height_sta,Month,season
index,,,,,,,,,,,,,,
0,279.28,0.0,91.4,200.0,3.05,277.97,14066001,2016-01-01 00:00:00,14066001_0_0,49.334,-0.431,2,1,winter
1,278.76,0.0,91.4,190.0,2.57,277.45,14066001,2016-01-01 01:00:00,14066001_0_1,49.334,-0.431,2,1,winter
2,278.27,0.0,91.7,181.0,2.26,277.02,14066001,2016-01-01 02:00:00,14066001_0_2,49.334,-0.431,2,1,winter
3,277.98,0.0,93.0,159.0,2.62,276.95,14066001,2016-01-01 03:00:00,14066001_0_3,49.334,-0.431,2,1,winter
4,277.32,0.0,95.9,171.0,2.99,276.72,14066001,2016-01-01 04:00:00,14066001_0_4,49.334,-0.431,2,1,winter


In [31]:
test['season'] = test['Month'].apply(season)

In [32]:
test.head()

,temp,precip,humidity,wind_direction,wind_speed,dew_point_temp,Id,number_sta,lat,lon,height_sta,day_index,Hour,Month,season
index,,,,,,,,,,,,,,,
0,278.35,0.13,92.23,191.75,4.66,277.18,14047002_277_4,14047002,49.275,-0.712,60,277,4,12,winter
1,278.40,0.00,91.18,184.79,4.79,277.06,14047002_277_5,14047002,49.275,-0.712,60,277,5,12,winter
2,279.01,0.00,91.80,183.92,4.60,277.79,14047002_277_6,14047002,49.275,-0.712,60,277,6,12,winter
3,279.66,0.00,93.41,179.11,5.17,278.65,14047002_277_7,14047002,49.275,-0.712,60,277,7,12,winter
4,279.99,0.00,94.04,181.35,5.86,279.10,14047002_277_8,14047002,49.275,-0.712,60,277,8,12,winter


## Computing VX and Vy 

In [33]:
def wind_cord(v,o):
    
    a = o*math.pi/180

    return v*math.cos(a), v*math.sin(a)
def wind_vx_vy(df):
  Vx =  []
  Vy = []
  df_dict = df.to_dict('records')
  for line in tqdm(df_dict):
        o = line['wind_direction']
        v = line['wind_speed']
        vx,vy = wind_cord(v,o)
        Vx.append(vx)
        Vy.append(vy)
  df["Vx"] = Vx
  df["Vy"] = Vy
  return df

In [34]:
test = wind_vx_vy(test)
train = wind_vx_vy(train)

100%|██████████| 4409474/4409474 [00:04<00:00, 1049210.42it/s]


In [35]:
train.head()

,temp,precip,humidity,wind_direction,wind_speed,dew_point_temp,number_sta,date,Id,lat,lon,height_sta,Month,season,Vx,Vy
index,,,,,,,,,,,,,,,,
0,279.28,0.0,91.4,200.0,3.05,277.97,14066001,2016-01-01 00:00:00,14066001_0_0,49.334,-0.431,2,1,winter,-2.866062,-1.043161
1,278.76,0.0,91.4,190.0,2.57,277.45,14066001,2016-01-01 01:00:00,14066001_0_1,49.334,-0.431,2,1,winter,-2.530956,-0.446276
2,278.27,0.0,91.7,181.0,2.26,277.02,14066001,2016-01-01 02:00:00,14066001_0_2,49.334,-0.431,2,1,winter,-2.259656,-0.039442
3,277.98,0.0,93.0,159.0,2.62,276.95,14066001,2016-01-01 03:00:00,14066001_0_3,49.334,-0.431,2,1,winter,-2.445981,0.938924
4,277.32,0.0,95.9,171.0,2.99,276.72,14066001,2016-01-01 04:00:00,14066001_0_4,49.334,-0.431,2,1,winter,-2.953188,0.467739


In [36]:
test.head()

,temp,precip,humidity,wind_direction,wind_speed,dew_point_temp,Id,number_sta,lat,lon,height_sta,day_index,Hour,Month,season,Vx,Vy
index,,,,,,,,,,,,,,,,,
0,278.35,0.13,92.23,191.75,4.66,277.18,14047002_277_4,14047002,49.275,-0.712,60,277,4,12,winter,-4.562352,-0.948971
1,278.40,0.00,91.18,184.79,4.79,277.06,14047002_277_5,14047002,49.275,-0.712,60,277,5,12,winter,-4.773271,-0.399984
2,279.01,0.00,91.80,183.92,4.60,277.79,14047002_277_6,14047002,49.275,-0.712,60,277,6,12,winter,-4.589238,-0.314472
3,279.66,0.00,93.41,179.11,5.17,278.65,14047002_277_7,14047002,49.275,-0.712,60,277,7,12,winter,-5.169376,0.080305
4,279.99,0.00,94.04,181.35,5.86,279.10,14047002_277_8,14047002,49.275,-0.712,60,277,8,12,winter,-5.858373,-0.138060


In [ ]:
train.to_parquet("/content/drive/MyDrive/meteo_france/train_stations_data/Data_preparation_results/first_eng_final_imputed_train_x_station.gzip", engine='fastparquet', compression='gzip')
test.to_parquet("/content/drive/MyDrive/meteo_france/train_stations_data/Data_preparation_results/first_eng_final_imputed_test_x_station.gzip", engine='fastparquet', compression='gzip')

################################################################################################################################################################################################################################################################################################################################################################################################################
############################################################

## EDA 

In [ ]:
!pip install sweetviz

In [37]:
import sweetviz as sv

In [ ]:
my_report  = sv.analyze([train,'Train'])
my_report.show_html('/content/drive/MyDrive/meteo_france/train_stations_data/Data_preparation_results/Train_report_after_first_eng.html')

                                             |          | [  0%]   00:00 -> (? left)

Report /content/drive/MyDrive/meteo_france/train_stations_data/Data_preparation_results/Train_report_after_first_eng.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [ ]:
my_report = sv.analyze([test, "Test Data"])
my_report.show_html('/content/drive/MyDrive/meteo_france/test_stations_data/Data_preparation_results/Test_report_after_first_eng.html')

                                             |          | [  0%]   00:00 -> (? left)

Report /content/drive/MyDrive/meteo_france/test_stations_data/Data_preparation_results/Test_report_after_first_eng.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [8]:
import pandas as pd
import numpy as np
import sklearn
from tqdm import tqdm 
import math
import os


train_path = 'C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/imputed_data/final_imputed_train_x_station.gzip'
test_path = 'C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/imputed_data/final_imputed_test_x_station.gzip'
month_id_path = 'C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/Test/X_stations/Id_month_test.csv'
output_data_path = 'C:/Users/alaek/DEFI_IA/france_meteo/Défi_IA/input/imputed_data/'




def Read_Parquet(chemin):
  return pd.read_parquet(chemin, engine='fastparquet')

def spliting(string, index):
    liste = string.split("_")
    return liste[index]


def season(month):
    if (month == 12 or 1 <= month <= 4):
        return "winter"   
    elif (4 <= month <= 5):
        return "spring" 
    elif (6 <= month <= 9):
        return "summer"
    else:
        return "fall"

def wind_cord(v,o):
    
    a = o*math.pi/180

    return v*math.cos(a), v*math.sin(a)


def wind_vx_vy(df):
  Vx =  []
  Vy = []
  df_dict = df.to_dict('records')
  for line in tqdm(df_dict):
        o = line['wind_direction']
        v = line['wind_speed']
        vx,vy = wind_cord(v,o)
        Vx.append(vx)
        Vy.append(vy)
  df["Vx"] = Vx
  df["Vy"] = Vy
  return df



In [10]:
if __name__=='__main__':
    
    train = Read_Parquet(train_path)
    test = Read_Parquet(test_path)
    month_id = pd.read_csv(month_id_path)


    train.height_sta = train.height_sta.astype('int16')


    test['Id'] = test['Id'].astype(str)
    test['day_index'] = test['Id'].apply(lambda x : spliting(x, 1))
    test['Hour'] = test['Id'].apply(lambda x : spliting(x, 2))
    test['number_sta'] = test['Id'].apply(lambda line : spliting(line,0)).astype('int64')
    test.Hour = test.Hour.astype('int16')
    test.height_sta = test.height_sta.astype('int16')

    month_id = month_id.astype(str)
    dictionnary = dict()
    for idx, val in month_id.values:
        dictionnary[idx] = val


    test['Month'] = test['day_index'].map(lambda x : dictionnary[x]).astype('int16')
    print(train.dtypes)

    train["Month"] = train["date"].dt.month.astype('int16')
    train["season"] = train['Month'].apply(season)

    test['season'] = test['Month'].apply(season)

    test = wind_vx_vy(test)
    train = wind_vx_vy(train)

    train.to_parquet(os.path.join(output_data_path ,"first_eng_final_imputed_train_x_station.gzip"), engine='fastparquet', compression='gzip')
    test.to_parquet(os.path.join(output_data_path ,"first_eng_final_imputed_test_x_station.gzip"), engine='fastparquet', compression='gzip')


temp                     float64
precip                   float64
humidity                 float64
wind_direction           float64
wind_speed               float64
dew_point_temp           float64
number_sta                 int64
date              datetime64[ns]
Id                        object
lat                      float64
lon                      float64
height_sta                 int16
dtype: object


100%|██████████| 4409474/4409474 [00:04<00:00, 1036778.13it/s]
